# Assigment 1:  Schema Matching

## Ground Truth

In [1]:
G = [('Imdb.Name', 'rt.Name'), ('Imdb.YearRange', 'rt.Year'), ('Imdb.ReleaseDate', 'rt.Release Date'), 
     ('Imdb.Director', 'rt.Director'), ('Imdb.Creator', 'rt.Creator'), ('Imdb.Cast', 'rt.Cast'),
    ('Imdb.Duration', 'rt.Duration'), ('Imdb.RatingValue', 'rt.RatingValue'), ('Imdb.Genre', 'rt.Genre'), 
     ('Imdb.Description', 'rt.Description')]
Num_of_all_correspondences = 10

In [2]:
import pandas as pd
import itertools
import numpy as np
file_path1 = 'imdb.csv'
file_path2 = 'rotten_tomatoes.csv'

# Read the csv file using pandas read_csv method.
data_frame1 = pd.read_csv(file_path1)
data_frame2 = pd.read_csv(file_path2)

## Task 1: Label-Based Schema Matching
Here, we want to find the correspondences between the columns from the two datasets with the help of only schema headers.

1. Provide an algorithm. Specify the input, output, and time complexity.
2. Implement the algorithm and report the results. 

In [15]:
import difflib
import re

schema_headers1 = list(data_frame1)
schema_headers2 = list(data_frame2)
lists = [schema_headers1, schema_headers2]

import Levenshtein

#reused code from https://stackoverflow.com/questions/3855537/fastest-way-to-sort-in-python
def qsort(inlist):
    if inlist == []: 
        return []
    else:
        pivot = inlist[0]
        lesser = qsort([x for x in inlist[1:] if x < pivot])
        greater = qsort([x for x in inlist[1:] if x >= pivot])
        return lesser + [pivot] + greater

def preprocessing(input_str):
    input_str = input_str.strip().lower()
    input_str = re.sub("\W", "", input_str).strip()
    return input_str

def distance(first_str, second_str):
    first_str = preprocessing(first_str)
    second_str = preprocessing(second_str)
    leven = Levenshtein.ratio(first_str, second_str)
    return leven

for a, b in itertools.combinations(lists, 2):
    result = []
    for first in a:
        for second in b:
            ratio=distance(first,second)  
            if ratio > 0.9:                  
                result.append(('Imdb.'+first,'rt.'+second))  
                break            
print(result)  

all_discovered_cor = len(result)
discovered_cor_in_G = len(list(set(result).intersection(G)))

['Id', 'Name', 'YearRange', 'ReleaseDate', 'Director', 'Creator', 'Cast', 'Duration', 'RatingValue', 'ContentRating', 'Genre', 'Url', 'Description']
['Id', 'Name', 'Year', 'Release Date', 'Director', 'Creator', 'Actors', 'Cast', 'Language', 'Country', 'Duration', 'RatingValue', 'RatingCount', 'ReviewCount', 'Genre', 'Filming Locations', 'Description']
[('Imdb.id', 'rt.id'), ('Imdb.name', 'rt.name'), ('Imdb.releasedate', 'rt.releasedate'), ('Imdb.director', 'rt.director'), ('Imdb.creator', 'rt.creator'), ('Imdb.cast', 'rt.cast'), ('Imdb.duration', 'rt.duration'), ('Imdb.ratingvalue', 'rt.ratingvalue'), ('Imdb.genre', 'rt.genre'), ('Imdb.description', 'rt.description')]


In [13]:
print("precision: ", discovered_cor_in_G/all_discovered_cor)
print("recall: ", discovered_cor_in_G/Num_of_all_correspondences)

precision:  0.0
recall:  0.0


<p> &nbsp;&nbsp;&nbsp;&nbsp; Is there any parameter that affects the results?</p>
<p> 3. What is the upsides and downsides of this method? When does it work and when
not?</p>

## Task 2: Instance-Based Schema Matching
Here, we want to find the correspondences between the columns from the two datasets with the help of only data values.
1. Provide an algorithm. Specify the input, output, and time complexity.
2. Implement the algorithm and report the results. 

In [18]:
import random
import time
import math
r = random.randint(0,len(data_frame1))
if(len(data_frame1) > len(data_frame2)):
    samples = int(math.ceil(len(data_frame1)/100))
    values1 = data_frame1._slice(slice(r-samples, r)).values
    values2 = data_frame2.values
    headers1= list(data_frame1)
    headers2= list(data_frame2)
else:
    samples = int(math.ceil(len(data_frame2)/100))
    values2 = data_frame2._slice(slice(r-samples, r)).values
    values1 = data_frame1.values
    
    headers1= list(data_frame1)
    headers2= list(data_frame2)

lists = [values1, values2]
    
t_a = time.clock()

for a, b in itertools.combinations(lists, 2):
    merged = []
    for first in itertools.chain.from_iterable(a):
        for second in itertools.chain.from_iterable(b):
            if(isinstance(first,str) and isinstance(second,str)):
                if(abs(len(first)-len(second)) < 1):
                    if distance(first, second) > 0.9:
                        i_a,j_a = np.where(a==first)
                        column_name1 = headers1[j_a[0]]
                        del headers1[j_a[0]]
                        i_b,j_b = np.where(b==second)
                        column_name2 = headers2[j_b[0]]
                        del headers2[j_b[0]]
                        match = ('Imdb.'+column_name2,'rt.'+column_name1)
                        merged.append(match)
                        a = np.delete(a, j_a, 1)
                        b = np.delete(b, j_b, 1)
                        break
            elif(isinstance(first, (int, float)) and isinstance(second, (int, float))):
                if pd.isnull(first):
                    break
                elif pd.isnull(second):
                    break
                else:
                    if abs(first-second) < 1:
                        i_a,j_a = np.where(a==first)
                        column_name1 = headers1[j_a[0]]
                        del headers1[j_a[0]]
                        i_b,j_b = np.where(b==second)
                        column_name2 = headers2[j_b[0]]
                        del headers2[j_b[0]]
                        match = ('Imdb.'+column_name2,'rt.'+column_name1)
                        merged.append(match)
                        a = np.delete(a, j_a, 1)
                        b = np.delete(b, j_b, 1)
                        break
                    
                    
print(merged)
t_b = time.clock()
total_time = t_b-t_a
print("run time:", total_time,"s")

all_discovered_cor = len(merged)
discovered_cor_in_G = len(list(set(merged).intersection(G)))

[('Imdb.Genre', 'rt.Genre'), ('Imdb.Director', 'rt.Director'), ('Imdb.Name', 'rt.Name'), ('Imdb.Creator', 'rt.Creator'), ('Imdb.Description', 'rt.Description')]
run time: 45.929708000000005 s


<p> &nbsp;&nbsp;&nbsp;&nbsp; Is there any parameter that affects the results?</p>
<p> 3. What is the upsides and downsides of this method? When does it work and when
not?</p>

In [19]:
print("precision: ", discovered_cor_in_G/all_discovered_cor)
print("recall: ", discovered_cor_in_G/Num_of_all_correspondences)

precision:  1.0
recall:  0.5
